In [1]:
!pip install findspark
!pip install pyspark
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=bbdda2958e64b5303f7a21857fbd896b58d93b57209447e5f086a648e8628e69
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 832.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=97fba94a5aaddc672b818979a755c5efe8f7daa77304f2206cc4a4231532c1f1
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [2]:
import pyspark
import findspark
import spark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [ ]:
lista = [n for n in range(1,101)]

for numero in lista:
    div3 = numero-((numero//3)*3)
    div5 = numero-((numero//5)*5)

    if div3 == 0 and div5 == 0:
        print('BI')
    elif div3 == 0:
        print('B')
    elif div5 == 0:
        print('I')
    else:
        print(numero)

# SERIE II

In [ ]:
cards = spark.read.option('inferSchema','true').option('header','true').csv('Credit Card Customer Data.csv')

In [ ]:
lower_column_name = [col.lower().replace(' ', '_') for col in cards.columns]
cards = cards.toDF(*lower_column_name)
cards.columns

['sl_no',
 'customer_key',
 'avg_credit_limit',
 'total_credit_cards',
 'total_visits_bank',
 'total_visits_online',
 'total_calls_made']

### 1. identificar al cliente con mayor limite de credito

In [ ]:
cards.select(
    'customer_key',
    'avg_credit_limit'
)\
.groupBy('customer_key').agg(max('avg_credit_limit').alias('max_credit_limit'))\
.orderBy(desc('max_credit_limit'))\
.limit(1)\
.show()

+------------+----------------+
|customer_key|max_credit_limit|
+------------+----------------+
|       33837|          200000|
+------------+----------------+



### 2. Basandose en el numero de visitas al banco, visitas online y total de llamadas. Crear un nuevo campo que pueda clasificar a los clientes de la siguiente manera:

  ##### a. Si el total de visitas al banco es mayor que otros = tradicional
  ##### b. Si el total de visitas online es mayor que otros = digital
  ##### c. Si el total de llamadas es mayor que otros = telefonico
  ##### d. Si tiene el mismo valor en dos o tres canales = Multicanal



In [ ]:
cards.select(
    'customer_key',
    'total_visits_bank',
    'total_visits_online',
    'total_calls_made',
    when((col('total_visits_bank') > col('total_visits_online')) & (col('total_visits_bank') > col('total_calls_made')), 'Tradicional').\
    when((col('total_visits_online') > col('total_visits_bank')) & (col('total_visits_online') > col('total_calls_made')), 'Digital').\
    when((col('total_calls_made') > col('total_visits_bank')) & (col('total_calls_made') > col('total_visits_online')), 'Telefonico').\
    otherwise('Multicanal').\
    alias('customer_type')
).show()

+------------+-----------------+-------------------+----------------+-------------+
|customer_key|total_visits_bank|total_visits_online|total_calls_made|customer_type|
+------------+-----------------+-------------------+----------------+-------------+
|       87073|                1|                  1|               0|   Multicanal|
|       38414|                0|                 10|               9|      Digital|
|       17341|                1|                  3|               4|   Telefonico|
|       40496|                1|                  1|               4|   Telefonico|
|       47437|                0|                 12|               3|      Digital|
|       58634|                0|                  1|               8|   Telefonico|
|       48370|                0|                 11|               2|      Digital|
|       37376|                0|                  1|               1|   Multicanal|
|       82490|                0|                  2|               2|   Mult

### 3. ¿Para qué podría utilizar una empresa del sector bancario el campo creado en el punto 2?

In [ ]:
'''
Para poder saber el medio favorito de interacción del cliente con el banco y por este, poder ofrecer nuevos productos o servicios.
Así como crear estrategias de mercado dependiendo la via favorita de los clientes en general.
'''

### 4. Sumar los campos total_visits_banck, total_visits_online, total_calls_mode para identificar a los 3 clientes con el mayor total.

In [ ]:
cards.select(
    'customer_key',
    (col('total_visits_bank') + col('total_visits_online') + col('total_calls_made')).alias('total_customer')
)\
.orderBy(desc('total_customer'))\
.limit(3)\
.show()

+------------+--------------+
|customer_key|total_customer|
+------------+--------------+
|       38414|            19|
|       99843|            18|
|       90839|            18|
+------------+--------------+



### 5. ¿Cuántas tarjetas de credito hay en total? ¿Cuántos clientes tiene la empresa? ¿Cuáñ es el límite total?

In [ ]:
cards.select(
    count('sl_no').alias('total_tarjetas'),
    count_distinct('customer_key').alias('conteo_clientes'),
    sum('avg_credit_limit').alias('total_limite')
).show()

+--------------+---------------+------------+
|total_tarjetas|conteo_clientes|total_limite|
+--------------+---------------+------------+
|           660|            655|    22819000|
+--------------+---------------+------------+



### 6. Crear una nueva estructura dataframe llamada: df_cc_filter con todos los clientes que tengan menos de 3 tarjetas de credito.

In [ ]:
df_cc_filter = cards.groupBy('customer_key').agg(count('customer_key').alias('customer_cards')).filter(col('customer_cards') < 3)
df_cc_filter.show()

+------------+--------------+
|customer_key|customer_cards|
+------------+--------------+
|       41946|             1|
|       48510|             1|
|       49331|             1|
|       64519|             1|
|       45341|             1|
|       39285|             1|
|       82164|             1|
|       34103|             1|
|       29058|             1|
|       18595|             1|
|       26767|             1|
|       44398|             1|
|       47496|             1|
|       97109|             1|
|       14377|             1|
|       34495|             1|
|       87219|             1|
|       90586|             1|
|       25742|             1|
|       17036|             1|
+------------+--------------+
only showing top 20 rows



# SERIE III

In [ ]:
'''

4. Identificar los dos productos con mas cantidades vendidas en cada año.
5. Encontrar la diferencia entre la venta y el costo de cda orden e ident
'''

### 1. Leer los archivos en su respectivo DF de pyspark y mostrar el esquema de cada DF

In [4]:
product = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Product.csv')
region = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Region.csv')
reseller = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Reseller.csv')
sales = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Sales.csv')
salesperson = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Salesperson.csv')
salespersonRegion = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('SalespersonRegion.csv')
targets = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Targets.csv')

In [ ]:
product.printSchema()

root
 |-- productkey: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- standard_cost: float (nullable = true)
 |-- color: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- category: string (nullable = true)
 |-- background_color_format: string (nullable = true)
 |-- font_color_format: string (nullable = true)



In [ ]:
region.printSchema()

root
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Group: string (nullable = true)



In [ ]:
reseller.printSchema()

root
 |-- ResellerKey: integer (nullable = true)
 |-- Business Type: string (nullable = true)
 |-- Reseller: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State-Province: string (nullable = true)
 |-- Country-Region: string (nullable = true)



In [ ]:
sales.printSchema()

root
 |-- SalesOrderNumber: string (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- ProductKey: integer (nullable = true)
 |-- ResellerKey: integer (nullable = true)
 |-- EmployeeKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Unit_Price: float (nullable = true)
 |-- Sales: float (nullable = true)
 |-- Cost: float (nullable = true)



In [ ]:
salesperson.printSchema()

root
 |-- EmployeeKey: integer (nullable = true)
 |-- EmployeeID: integer (nullable = true)
 |-- Salesperson: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- UPN: string (nullable = true)



In [ ]:
salespersonRegion.printSchema()

root
 |-- EmployeeKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)



In [ ]:
targets.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- Target: string (nullable = true)
 |-- TargetMonth: date (nullable = true)



### 2. Realizar el proceso de limpieza de datos en caso de ser necesario.

In [5]:
def format_column_names(columns):
  return [col.replace(' ', '_').replace('-','_') for col in columns]

In [6]:
# Aplicando formato a columnas
product = product.toDF(*format_column_names(product.columns))

# Cast correcto a columnas
product = product.select(
    col('productkey').cast('int').alias('productkey'),
    'product',
    regexp_extract('standard_cost', r'([0-9]+).([0-9]+)', 0).cast('float').alias('standard_cost'),
    'color',
    'subcategory',
    'category',
    'background_color_format',
    'font_color_format'
)

In [7]:
region = region.select(
    col('SalesTerritoryKey').cast('int'),
    'Region',
    'Country',
    'Group'
)

region = region.toDF(*format_column_names(region.columns))

In [8]:
reseller = reseller.toDF(*format_column_names(reseller.columns))

In [10]:
# Función que convierte fecha en letras a numeros en string para poder aplicar un cast tipo date

@udf(returnType=StringType())
def date_to_str_fun(date):
    months = ['january','february','march','april','may','june','july','august','september','october','november','december']

    a = date.split(', ')
    b = [x.split(' ') for x in a[1:]]
    year_var = a[2]
    month_var = months.index(b[0][0].lower())+1
    day_var = b[0][1]

    if month_var < 10: month_var = f'0{str(month_var)}'
    if int(day_var) < 10: day_var = f'0{day_var}'

    return f'{str(year_var)}-{str(month_var)}-{str(day_var)}'

In [11]:
sales = sales.toDF(*format_column_names(sales.columns))

# Nuevo DF sales con formato correcto a los datos
sales = sales.select(
    'SalesOrderNumber',
    date_to_str_fun('OrderDate').alias('OrderDate'),
    'ProductKey',
    'ResellerKey',
    'EmployeeKey',
    'SalesTerritoryKey',
    'Quantity',
    regexp_extract(regexp_replace('Unit_Price', ',', ''), r'([0-9]+).([0-9]+)', 0).cast('float').alias('Unit_Price'),
    regexp_extract(regexp_replace('Sales', ',', ''), r'([0-9]+).([0-9]+)', 0).cast('float').alias('Sales'),
    regexp_extract(regexp_replace('Cost', ',', ''), r'([0-9]+).([0-9]+)', 0).cast('float').alias('Cost')
)

In [13]:
targets = targets.select(
    'EmployeeID',
    regexp_extract(regexp_replace('Target',',',''), r'([0-9]+).([0-9]+)',0).alias('Target'),
    date_to_str_fun('TargetMonth').alias('TargetMonth')
)

### 3. mostrar las ventas por región comparado anualmente nombre_region, año 2017, 2018, 2019, 2020

In [17]:
sales_region = sales.join(region, 'SalesTerritoryKey')

In [19]:
sales_region\
.select(
    'region',
    year('OrderDate').alias('year'),
    'Sales'
)\
.groupBy('Region', 'year').agg(sum('Sales').alias('total_sales'))\
.show()

+--------------+----+------------------+
|        Region|year|       total_sales|
+--------------+----+------------------+
|        Canada|2019| 5689304.063606858|
|       Central|2017| 951799.9631867409|
|United Kingdom|2019|2186342.9776256084|
|     Southeast|2017|1450813.3120913506|
|United Kingdom|2020| 852025.0148931742|
|     Northwest|2018|3502111.0230522156|
|     Australia|2020| 515331.0184943676|
|     Northwest|2020| 2124429.429153204|
|United Kingdom|2018| 843795.9444041252|
|     Northwest|2017|1690237.4899878502|
|     Southwest|2020| 2587513.102998495|
|       Central|2018|2635029.3197636604|
|     Northeast|2017| 568552.2762298584|
|       Central|2019| 3015528.528031826|
|     Southeast|2020| 904561.4563504457|
|     Northwest|2019| 4686781.475811243|
|     Southeast|2019| 2443724.734798074|
|     Southwest|2018| 6329568.597466469|
|        Canada|2018|4860350.5877485275|
|     Northeast|2020| 816863.3455300331|
+--------------+----+------------------+
only showing top